In [ ]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt
import pandas as pd
import numpy as np


# Strategy

In [ ]:
class TestStrategy(bt.Strategy):
    params = (
        ('fast_maperiod', 50),
        ('slow_maperiod', 200),
        ('printlog', False),
    )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function for this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None
        
        self.fast_sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.fast_maperiod)
        
        self.slow_sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.slow_maperiod)
        
        # Indicators for the plotting show
        bt.indicators.ExponentialMovingAverage(self.datas[0], period=25)
        bt.indicators.WeightedMovingAverage(self.datas[0], period=25,
                                            subplot=True)
        bt.indicators.StochasticSlow(self.datas[0])
        bt.indicators.MACDHisto(self.datas[0])
        rsi = bt.indicators.RSI(self.datas[0])
        bt.indicators.SmoothedMovingAverage(rsi, period=10)
        bt.indicators.ATR(self.datas[0], plot=False)
        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if (self.dataclose[0] > self.fast_sma[0]) and (self.dataclose[0] > self.dataclose[-1]) and (self.fast_sma[0] > self.slow_sma[0]):

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:

            # Already in the market ... we might sell
             if self.dataclose[0] < self.fast_sma[0] and (self.fast_sma[0] < self.slow_sma[0]):
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()
                
    def stop(self):
        self.log('(Fast MA Period %2d) (Slow MA Period %2d) Ending Value %.2f' %
                 (self.params.fast_maperiod, self.params.slow_maperiod,self.broker.getvalue()), doprint=True)

# Analyzer

In [ ]:
class myAnalyzer(bt.Analyzer):
    
    def __init__(self):
        
        self.fast_period = self.strategy.params.fast_maperiod
        self.slow_period = self.strategy.params.slow_maperiod
        
    def start(self):
        self.open_pos = self.strategy.broker.get_value() 
        self.trades = 0

    def notify_trade(self, trade):
        self.trades += 1
    
    def stop(self):
        self.close_pos = self.strategy.broker.get_value()

    def result(self):
        
        output = {
            'params': (self.fast_period, self.slow_period),
            'profit': self.close_pos - self.open_pos,
            'trades': self.trades
        }
        
        return output

# Optimization


In [ ]:
if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()
    
    strats = cerebro.optstrategy(
        TestStrategy,
        fast_maperiod=(5,8,10,12,15,20,25,30),
        slow_maperiod=(30,40,50,60,70,100,150,200))

    # Datas are in a subfolder of the samples. Need to find where the script is
    # because it could have been called from anywhere
    datapath = os.path.join('../data/raw/demo/orcl-1995-2014.txt')

    # Create a Data Feed
    # Load data
    fname_symbol = 'CL'
    folder_name = '5min'
    suffix = '5min_20160103_20190405'

    df = pd.read_parquet(os.path.join('../data/processed/{}/'.format(folder_name), '{}_{}.parquet'.format(fname_symbol, suffix)))
    df = (df.resample('4h', label='left', base=18).agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'}))
    df.columns = [col_name.lower() for col_name in df.columns]
    df = df.dropna()

    data = bt.feeds.PandasData(dataname = df['2017-01-01':'2019-01-01'])
    
    cerebro.addanalyzer(myAnalyzer, _name='myAnalyzer')

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)
    
    # Add a FixedSize sizer according to the stake
    cerebro.addsizer(bt.sizers.FixedSize, stake=10)
    
    cerebro.broker.set_slippage_fixed(.01)
    cerebro.broker.setcommission(commission=0.001)

    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    results = cerebro.run(maxcpus=1)

    # Print out the final result

# Analyze

In [ ]:
params  = []
trades = []
profit   = []
for res in results:
    r = res[0].analyzers.myAnalyzer.result()
    params.append(r['params'])
    trades.append(r['trades'])
    profit.append(r['profit'])


prof_ind = np.argmax(profit)
best_params = params[prof_ind]
print("Optimized: fast sma period, slow sma period: ",best_params,"\nTotal Profit: ", profit[prof_ind], "\nNumber of trades for the parameters pair: ", trades[prof_ind])

# Out of sample

In [ ]:
cerebro = bt.Cerebro()

cerebro.addstrategy(TestStrategy, fast_maperiod=15, slow_maperiod=50)
data = bt.feeds.PandasData(dataname = df['2016-01-01':'2017-01-01'])
    
cerebro.addanalyzer(myAnalyzer, _name='myAnalyzer')

    # Add the Data Feed to Cerebro
cerebro.adddata(data)

    # Set our desired cash start
cerebro.broker.setcash(100000.0)
    
    # Add a FixedSize sizer according to the stake
cerebro.addsizer(bt.sizers.FixedSize, stake=10)
    
cerebro.broker.set_slippage_fixed(.01)
cerebro.broker.setcommission(commission=0.001)

    # Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
results = cerebro.run(maxcpus=1)

print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

In [ ]:
# MISSING SHARPE / MISSING IS/OOS PERFORMANCE